In [1]:
import pandas as pd
from datetime import datetime,timedelta
import requests
import json

In [2]:
# Binance
def getCandles(symbol, interval, limit=0, startTime=None, endTime=None):
    url = 'https://www.binance.co/api/v1/klines?'
    params = {
        'symbol': symbol,
        'interval': interval
    }
    if limit:
        params['limit'] = limit
    if startTime:
        params['startTime'] = int(startTime.timestamp()*1000)
    if endTime:
        params['endTime'] = int(endTime.timestamp()*1000)
    r = requests.get(url, headers={
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'application/json'
        }, params = params,timeout=300)
    text = json.loads(r.content)
    df = pd.DataFrame(text, columns=[
        "opentime", "open", "high", "low", "close", "volume","closetime","Quote","trades","buybase","buyquote","ignore"])
    df["datetime"] = df["opentime"].map(lambda x: datetime.fromtimestamp(x / 1000).strftime("%Y%m%d %H:%M:%S"))
    return df
# interval available: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

In [3]:
symbol = 'BTCUSDT'
EXCHANGE = "binance"
def vt_symbol(symbol):
    return "%s:%s" % (symbol, EXCHANGE)
def write(collection, data):
    for index, row in data.iterrows():
        bar = {}
        bar['open'] = float(row.open)
        bar['close'] = float(row.close)
        bar['high'] = float(row.high)
        bar['low'] = float(row.low)
        bar['volume'] = float(row.volume)
        bar['symbol'] = symbol
        bar['exchange'] = EXCHANGE
        bar['vtSymbol'] = vt_symbol(symbol)
        bar['openInterest'] = 0
        bar['datetime'] = datetime.strptime(row.datetime, "%Y%m%d %H:%M:%S")
        bar['date'] = bar['datetime'].date().strftime('%Y%m%d')
        bar['time'] = bar['datetime'].time().strftime('%H:%M:%S')
        flt = {'datetime': bar['datetime']}
        collection.update_one(flt, {'$set':bar}, upsert=True)
    
from datetime import timedelta
def iter_dates(start, end):
    start = start.replace(second=0, microsecond=0)
    while start < end:
        _next = start + timedelta(hours=12)
        yield start, _next - timedelta(seconds=1)
        start = _next

In [ ]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)
collection = client['VnTrader_1Min_Db'][vt_symbol(symbol)]
collection.create_index([('datetime', pymongo.ASCENDING)], unique=True)
for s, e in iter_dates(datetime(2018, 1, 1), datetime(2018, 12, 10)):
    data = getCandles(symbol,'1m' , 1000,startTime=s, endTime=e)
    write(collection, data)
    print(symbol, s, e, data.shape)

BTCUSDT 2018-01-01 00:00:00 2018-01-01 11:59:59 (720, 13)
BTCUSDT 2018-01-01 12:00:00 2018-01-01 23:59:59 (720, 13)
BTCUSDT 2018-01-02 00:00:00 2018-01-02 11:59:59 (720, 13)
BTCUSDT 2018-01-02 12:00:00 2018-01-02 23:59:59 (720, 13)
BTCUSDT 2018-01-03 00:00:00 2018-01-03 11:59:59 (720, 13)
BTCUSDT 2018-01-03 12:00:00 2018-01-03 23:59:59 (720, 13)
BTCUSDT 2018-01-04 00:00:00 2018-01-04 11:59:59 (661, 13)
BTCUSDT 2018-01-04 12:00:00 2018-01-04 23:59:59 (654, 13)
BTCUSDT 2018-01-05 00:00:00 2018-01-05 11:59:59 (720, 13)
BTCUSDT 2018-01-05 12:00:00 2018-01-05 23:59:59 (720, 13)
BTCUSDT 2018-01-06 00:00:00 2018-01-06 11:59:59 (720, 13)
BTCUSDT 2018-01-06 12:00:00 2018-01-06 23:59:59 (720, 13)
BTCUSDT 2018-01-07 00:00:00 2018-01-07 11:59:59 (720, 13)
BTCUSDT 2018-01-07 12:00:00 2018-01-07 23:59:59 (720, 13)
BTCUSDT 2018-01-08 00:00:00 2018-01-08 11:59:59 (720, 13)
BTCUSDT 2018-01-08 12:00:00 2018-01-08 23:59:59 (720, 13)
BTCUSDT 2018-01-09 00:00:00 2018-01-09 11:59:59 (720, 13)
BTCUSDT 2018-0

BTCUSDT 2018-03-13 00:00:00 2018-03-13 11:59:59 (720, 13)
BTCUSDT 2018-03-13 12:00:00 2018-03-13 23:59:59 (720, 13)
BTCUSDT 2018-03-14 00:00:00 2018-03-14 11:59:59 (720, 13)
BTCUSDT 2018-03-14 12:00:00 2018-03-14 23:59:59 (720, 13)
BTCUSDT 2018-03-15 00:00:00 2018-03-15 11:59:59 (720, 13)
BTCUSDT 2018-03-15 12:00:00 2018-03-15 23:59:59 (720, 13)
BTCUSDT 2018-03-16 00:00:00 2018-03-16 11:59:59 (720, 13)
BTCUSDT 2018-03-16 12:00:00 2018-03-16 23:59:59 (720, 13)
BTCUSDT 2018-03-17 00:00:00 2018-03-17 11:59:59 (720, 13)
BTCUSDT 2018-03-17 12:00:00 2018-03-17 23:59:59 (720, 13)
BTCUSDT 2018-03-18 00:00:00 2018-03-18 11:59:59 (720, 13)
BTCUSDT 2018-03-18 12:00:00 2018-03-18 23:59:59 (720, 13)
BTCUSDT 2018-03-19 00:00:00 2018-03-19 11:59:59 (720, 13)
BTCUSDT 2018-03-19 12:00:00 2018-03-19 23:59:59 (720, 13)
BTCUSDT 2018-03-20 00:00:00 2018-03-20 11:59:59 (720, 13)
BTCUSDT 2018-03-20 12:00:00 2018-03-20 23:59:59 (720, 13)
BTCUSDT 2018-03-21 00:00:00 2018-03-21 11:59:59 (720, 13)
BTCUSDT 2018-0

BTCUSDT 2018-05-23 00:00:00 2018-05-23 11:59:59 (720, 13)
BTCUSDT 2018-05-23 12:00:00 2018-05-23 23:59:59 (720, 13)
BTCUSDT 2018-05-24 00:00:00 2018-05-24 11:59:59 (720, 13)
BTCUSDT 2018-05-24 12:00:00 2018-05-24 23:59:59 (720, 13)
BTCUSDT 2018-05-25 00:00:00 2018-05-25 11:59:59 (720, 13)
BTCUSDT 2018-05-25 12:00:00 2018-05-25 23:59:59 (720, 13)
BTCUSDT 2018-05-26 00:00:00 2018-05-26 11:59:59 (720, 13)
BTCUSDT 2018-05-26 12:00:00 2018-05-26 23:59:59 (720, 13)
BTCUSDT 2018-05-27 00:00:00 2018-05-27 11:59:59 (720, 13)
BTCUSDT 2018-05-27 12:00:00 2018-05-27 23:59:59 (720, 13)
BTCUSDT 2018-05-28 00:00:00 2018-05-28 11:59:59 (720, 13)
BTCUSDT 2018-05-28 12:00:00 2018-05-28 23:59:59 (720, 13)
BTCUSDT 2018-05-29 00:00:00 2018-05-29 11:59:59 (720, 13)
BTCUSDT 2018-05-29 12:00:00 2018-05-29 23:59:59 (720, 13)
BTCUSDT 2018-05-30 00:00:00 2018-05-30 11:59:59 (720, 13)
BTCUSDT 2018-05-30 12:00:00 2018-05-30 23:59:59 (720, 13)
BTCUSDT 2018-05-31 00:00:00 2018-05-31 11:59:59 (720, 13)
BTCUSDT 2018-0

BTCUSDT 2018-08-02 00:00:00 2018-08-02 11:59:59 (720, 13)
BTCUSDT 2018-08-02 12:00:00 2018-08-02 23:59:59 (720, 13)
BTCUSDT 2018-08-03 00:00:00 2018-08-03 11:59:59 (720, 13)
BTCUSDT 2018-08-03 12:00:00 2018-08-03 23:59:59 (720, 13)
BTCUSDT 2018-08-04 00:00:00 2018-08-04 11:59:59 (720, 13)
BTCUSDT 2018-08-04 12:00:00 2018-08-04 23:59:59 (720, 13)
BTCUSDT 2018-08-05 00:00:00 2018-08-05 11:59:59 (720, 13)
BTCUSDT 2018-08-05 12:00:00 2018-08-05 23:59:59 (720, 13)
BTCUSDT 2018-08-06 00:00:00 2018-08-06 11:59:59 (720, 13)
BTCUSDT 2018-08-06 12:00:00 2018-08-06 23:59:59 (720, 13)
BTCUSDT 2018-08-07 00:00:00 2018-08-07 11:59:59 (720, 13)
BTCUSDT 2018-08-07 12:00:00 2018-08-07 23:59:59 (720, 13)
BTCUSDT 2018-08-08 00:00:00 2018-08-08 11:59:59 (720, 13)
BTCUSDT 2018-08-08 12:00:00 2018-08-08 23:59:59 (720, 13)
BTCUSDT 2018-08-09 00:00:00 2018-08-09 11:59:59 (720, 13)
BTCUSDT 2018-08-09 12:00:00 2018-08-09 23:59:59 (720, 13)
BTCUSDT 2018-08-10 00:00:00 2018-08-10 11:59:59 (720, 13)
BTCUSDT 2018-0

BTCUSDT 2018-10-12 00:00:00 2018-10-12 11:59:59 (720, 13)
BTCUSDT 2018-10-12 12:00:00 2018-10-12 23:59:59 (720, 13)
BTCUSDT 2018-10-13 00:00:00 2018-10-13 11:59:59 (720, 13)
BTCUSDT 2018-10-13 12:00:00 2018-10-13 23:59:59 (720, 13)
BTCUSDT 2018-10-14 00:00:00 2018-10-14 11:59:59 (720, 13)
BTCUSDT 2018-10-14 12:00:00 2018-10-14 23:59:59 (720, 13)
BTCUSDT 2018-10-15 00:00:00 2018-10-15 11:59:59 (720, 13)
BTCUSDT 2018-10-15 12:00:00 2018-10-15 23:59:59 (720, 13)
BTCUSDT 2018-10-16 00:00:00 2018-10-16 11:59:59 (720, 13)
BTCUSDT 2018-10-16 12:00:00 2018-10-16 23:59:59 (720, 13)
BTCUSDT 2018-10-17 00:00:00 2018-10-17 11:59:59 (720, 13)
BTCUSDT 2018-10-17 12:00:00 2018-10-17 23:59:59 (720, 13)
BTCUSDT 2018-10-18 00:00:00 2018-10-18 11:59:59 (720, 13)
BTCUSDT 2018-10-18 12:00:00 2018-10-18 23:59:59 (720, 13)
BTCUSDT 2018-10-19 00:00:00 2018-10-19 11:59:59 (720, 13)
BTCUSDT 2018-10-19 12:00:00 2018-10-19 23:59:59 (510, 13)
BTCUSDT 2018-10-20 00:00:00 2018-10-20 11:59:59 (720, 13)
BTCUSDT 2018-1